In [5]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, MultiLabelBinarizer

import re

from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [6]:
main_df = pd.read_csv('DB/Diseases.csv')

In [7]:
main_df.head(10)

,Disease,Symptom_1,Symptom_2,Symptom_3,Symptom_4,Symptom_5,Symptom_6,Symptom_7,Symptom_8,Symptom_9,Symptom_10,Symptom_11,Symptom_12,Symptom_13,Symptom_14,Symptom_15,Symptom_16,Symptom_17
0,Allergy,continuous_sneezing,shivering,chills,watering_from_eyes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Allergy,shivering,chills,watering_from_eyes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Allergy,continuous_sneezing,chills,watering_from_eyes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Allergy,continuous_sneezing,shivering,watering_from_eyes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Allergy,continuous_sneezing,shivering,chills,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Common Cold,continuous_sneezing,chills,fatigue,cough,fever,headache,swelled_lymph_nodes,malaise,phlegm,throat_irritation,redness_of_eyes,sinus_pressure,runny_nose,congestion,chest_pain,loss_of_smell,muscle_pain
6,Common Cold,continuous_sneezing,chills,fatigue,cough,fever,headache,swelled_lymph_nodes,malaise,phlegm,throat_irritation,redness_of_eyes,sinus_pressure,runny_nose,congestion,chest_pain,loss_of_smell,muscle_pain
7,Common Cold,chills,fatigue,cough,fever,headache,swelled_lymph_nodes,malaise,phlegm,throat_irritation,redness_of_eyes,sinus_pressure,runny_nose,congestion,chest_pain,loss_of_smell,muscle_pain,NaN
8,Common Cold,continuous_sneezing,fatigue,cough,fever,headache,swelled_lymph_nodes,malaise,phlegm,throat_irritation,redness_of_eyes,sinus_pressure,runny_nose,congestion,chest_pain,loss_of_smell,muscle_pain,NaN
9,Common Cold,continuous_sneezing,chills,cough,fever,headache,swelled_lymph_nodes,malaise,phlegm,throat_irritation,redness_of_eyes,sinus_pressure,runny_nose,congestion,chest_pain,loss_of_smell,muscle_pain,NaN


In [8]:
df = main_df.copy()

In [9]:
#df.dropna(axis=1, how='all', inplace=True)

In [10]:
df.isnull().sum()

Disease         0
Symptom_1       0
Symptom_2       0
Symptom_3      16
Symptom_4      22
Symptom_5      17
Symptom_6      31
Symptom_7      56
Symptom_8      78
Symptom_9     114
Symptom_10    133
Symptom_11    153
Symptom_12    164
Symptom_13    172
Symptom_14    175
Symptom_15    177
Symptom_16    177
Symptom_17    186
dtype: int64

In [11]:
df.fillna(method='ffill', inplace=True)

C:\Users\Hp\AppData\Local\Temp\ipykernel_11284\3970806690.py:1: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='ffill', inplace=True)


In [12]:
df.head()

,Disease,Symptom_1,Symptom_2,Symptom_3,Symptom_4,Symptom_5,Symptom_6,Symptom_7,Symptom_8,Symptom_9,Symptom_10,Symptom_11,Symptom_12,Symptom_13,Symptom_14,Symptom_15,Symptom_16,Symptom_17
0,Allergy,continuous_sneezing,shivering,chills,watering_from_eyes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Allergy,shivering,chills,watering_from_eyes,watering_from_eyes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Allergy,continuous_sneezing,chills,watering_from_eyes,watering_from_eyes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Allergy,continuous_sneezing,shivering,watering_from_eyes,watering_from_eyes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Allergy,continuous_sneezing,shivering,chills,watering_from_eyes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
class CustomLabelEncoder(LabelEncoder):
    def __init__(self, start = 0):
        self.start = start
        super().__init__()

    def fit_transform(self, y):
        encoded = super().fit_transform(y)
        encoded += self.start
        return encoded

In [14]:
flasttened_series = df['Disease'].astype(str)

In [15]:
flasttened_series

0       Allergy
1       Allergy
2       Allergy
3       Allergy
4       Allergy
         ...   
183    Migraine
184    Migraine
185    Migraine
186    Migraine
187    Migraine
Name: Disease, Length: 188, dtype: object

In [16]:
encoder = CustomLabelEncoder(start=200)

In [17]:
encoded_values = encoder.fit_transform(flasttened_series)

In [18]:
encoded_values

array([200, 200, 200, 200, 200, 206, 206, 206, 206, 206, 206, 206, 206,
       206, 206, 208, 208, 208, 208, 208, 208, 208, 209, 209, 209, 209,
       209, 209, 203, 203, 203, 203, 203, 203, 203, 210, 210, 210, 210,
       210, 210, 210, 210, 210, 210, 210, 210, 210, 210, 204, 204, 204,
       204, 204, 204, 204, 204, 204, 204, 204, 204, 204, 218, 218, 218,
       218, 218, 218, 218, 218, 218, 218, 218, 218, 218, 218, 218, 218,
       215, 215, 215, 215, 215, 215, 215, 215, 215, 211, 211, 211, 211,
       211, 211, 211, 211, 201, 201, 201, 201, 201, 201, 201, 201, 213,
       213, 213, 213, 213, 213, 213, 219, 219, 219, 219, 219, 219, 219,
       219, 216, 216, 216, 216, 216, 216, 216, 216, 205, 205, 205, 205,
       205, 205, 205, 205, 205, 205, 205, 205, 205, 202, 202, 202, 202,
       217, 217, 217, 217, 217, 217, 217, 217, 217, 217, 217, 217, 207,
       207, 207, 207, 207, 207, 207, 207, 207, 207, 207, 207, 207, 207,
       207, 207, 212, 212, 212, 212, 212, 212, 212, 212, 212, 21

In [19]:
df['Disease'] = encoded_values

In [20]:
mapping_data = {'label_encoder' : encoder}

In [21]:
mapping_data

{'label_encoder': CustomLabelEncoder(start=200)}

In [22]:
label_mapping = {k : v for k, v in zip(mapping_data['label_encoder'].classes_, range(200, 200 + len(mapping_data['label_encoder'].classes_)))}

In [23]:
label_mapping

{'Allergy': 200,
 'Arthritis': 201,
 'Autoimmune Diseases': 202,
 'Bronchial Asthma': 203,
 'Chicken pox': 204,
 'Chronic Kidney Disease': 205,
 'Common Cold': 206,
 'Coronary Artery Disease': 207,
 'Diabetes ': 208,
 'Gastroenteritis': 209,
 'Hypertension': 210,
 'Hypothyroidism': 211,
 'Inflammatory Bowel Disease (IBD)': 212,
 'Lower Urinary tract infection': 213,
 'Migraine': 214,
 'Pneumonia': 215,
 'Psoriasis': 216,
 'Stroke': 217,
 'Tuberculosis': 218,
 'Upper Urinary tract infection': 219}

In [24]:
df = main_df.copy()

In [25]:
df['All Symptoms'] = df.apply(lambda row: ','.join(row.dropna()), axis=1)

In [26]:
df['All Symptoms']

0      Allergy, continuous_sneezing, shivering, chill...
1         Allergy, shivering, chills, watering_from_eyes
2      Allergy, continuous_sneezing, chills, watering...
3      Allergy, continuous_sneezing, shivering, water...
4        Allergy, continuous_sneezing, shivering, chills
                             ...                        
183    Migraine,stiff_neck,excessive_hunger,visual_di...
184    Migraine,stiff_neck,excessive_hunger,visual_di...
185    Migraine,stiff_neck,excessive_hunger,visual_di...
186    Migraine,stiff_neck,excessive_hunger,visual_di...
187    Migraine,stiff_neck,excessive_hunger,visual_di...
Name: All Symptoms, Length: 188, dtype: object

In [27]:
df['All Symptoms'] = df['All Symptoms'].apply(lambda x: ','.join(sorted(set(x.split(','))) if x else ''))

In [28]:
df['All Symptoms']

0       chills, continuous_sneezing, shivering, water...
1          chills, shivering, watering_from_eyes,Allergy
2       chills, continuous_sneezing, watering_from_ey...
3       continuous_sneezing, shivering, watering_from...
4         chills, continuous_sneezing, shivering,Allergy
                             ...                        
183    Migraine,blurred_and_distorted_vision,difficul...
184    Migraine,blurred_and_distorted_vision,dizzines...
185    Migraine,blurred_and_distorted_vision,changes_...
186    Migraine,blurred_and_distorted_vision,changes_...
187    Migraine,blurred_and_distorted_vision,changes_...
Name: All Symptoms, Length: 188, dtype: object

In [29]:
stay_cols= ['Disease', 'All Symptoms']

In [30]:
df = df[stay_cols]

In [31]:
df.head()

,Disease,All Symptoms
0,Allergy,"chills, continuous_sneezing, shivering, water..."
1,Allergy,"chills, shivering, watering_from_eyes,Allergy"
2,Allergy,"chills, continuous_sneezing, watering_from_ey..."
3,Allergy,"continuous_sneezing, shivering, watering_from..."
4,Allergy,"chills, continuous_sneezing, shivering,Allergy"


In [32]:
def strip_to_basic_tokens(text):
    text = re.sub(r'[_\s]+', ' ', text)

    tokens = [token.strip().lower() for token in text.split(',')]
    return tokens

In [33]:
df['Basic Tokens'] = df['All Symptoms'].apply(strip_to_basic_tokens)
df['Basic Tokens'] = df['Basic Tokens'].apply(lambda x: ', '.join(x))

In [34]:
df.head()

,Disease,All Symptoms,Basic Tokens
0,Allergy,"chills, continuous_sneezing, shivering, water...","chills, continuous sneezing, shivering, wateri..."
1,Allergy,"chills, shivering, watering_from_eyes,Allergy","chills, shivering, watering from eyes, allergy"
2,Allergy,"chills, continuous_sneezing, watering_from_ey...","chills, continuous sneezing, watering from eye..."
3,Allergy,"continuous_sneezing, shivering, watering_from...","continuous sneezing, shivering, watering from ..."
4,Allergy,"chills, continuous_sneezing, shivering,Allergy","chills, continuous sneezing, shivering, allergy"


In [35]:
df = df.drop(['All Symptoms'], axis=1)

In [67]:
df.head()

,Disease,Basic Tokens
0,Allergy,"chills, continuous sneezing, shivering, wateri..."
1,Allergy,"chills, shivering, watering from eyes, allergy"
2,Allergy,"chills, continuous sneezing, watering from eye..."
3,Allergy,"continuous sneezing, shivering, watering from ..."
4,Allergy,"chills, continuous sneezing, shivering, allergy"


In [37]:
dfE = df.copy()

In [38]:
dfE['Basic Tokens']

0      chills, continuous sneezing, shivering, wateri...
1         chills, shivering, watering from eyes, allergy
2      chills, continuous sneezing, watering from eye...
3      continuous sneezing, shivering, watering from ...
4        chills, continuous sneezing, shivering, allergy
                             ...                        
183    migraine, blurred and distorted vision, diffic...
184    migraine, blurred and distorted vision, dizzin...
185    migraine, blurred and distorted vision, change...
186    migraine, blurred and distorted vision, change...
187    migraine, blurred and distorted vision, change...
Name: Basic Tokens, Length: 188, dtype: object

In [39]:
dfE['Basic Tokens'] = dfE['Basic Tokens'].apply(lambda x: x.split(', '))

In [40]:
dfE['Basic Tokens']

0      [chills, continuous sneezing, shivering, water...
1       [chills, shivering, watering from eyes, allergy]
2      [chills, continuous sneezing, watering from ey...
3      [continuous sneezing, shivering, watering from...
4      [chills, continuous sneezing, shivering, allergy]
                             ...                        
183    [migraine, blurred and distorted vision, diffi...
184    [migraine, blurred and distorted vision, dizzi...
185    [migraine, blurred and distorted vision, chang...
186    [migraine, blurred and distorted vision, chang...
187    [migraine, blurred and distorted vision, chang...
Name: Basic Tokens, Length: 188, dtype: object

In [41]:
mlb = MultiLabelBinarizer()

In [42]:
one_hot_encoded = pd.DataFrame(mlb.fit_transform(dfE['Basic Tokens']), columns=mlb.classes_, index=df.index)

In [43]:
one_hot_encoded

,abnormal heart rhythm,abnormal menstruation,aching body,agitation,allergy,angina,anxiety,arm paralysis,arthritis,autoimmune diseases,...,upset stomach,vision impairment,visual disturbances,vomiting,warm red skin over the affected joint,watering from eyes,weight gain,weight loss,wheezing,yellowing of eyes
0,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
184,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
185,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
186,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0


In [44]:
df_encoded = pd.concat([dfE, one_hot_encoded], axis=1)

In [45]:
df_encoded

,Disease,Basic Tokens,abnormal heart rhythm,abnormal menstruation,aching body,agitation,allergy,angina,anxiety,arm paralysis,...,upset stomach,vision impairment,visual disturbances,vomiting,warm red skin over the affected joint,watering from eyes,weight gain,weight loss,wheezing,yellowing of eyes
0,Allergy,"[chills, continuous sneezing, shivering, water...",0,0,0,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1,Allergy,"[chills, shivering, watering from eyes, allergy]",0,0,0,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,Allergy,"[chills, continuous sneezing, watering from ey...",0,0,0,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3,Allergy,"[continuous sneezing, shivering, watering from...",0,0,0,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,Allergy,"[chills, continuous sneezing, shivering, allergy]",0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183,Migraine,"[migraine, blurred and distorted vision, diffi...",0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
184,Migraine,"[migraine, blurred and distorted vision, dizzi...",0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
185,Migraine,"[migraine, blurred and distorted vision, chang...",0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
186,Migraine,"[migraine, blurred and distorted vision, chang...",0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0


In [46]:
df_encoded = df_encoded.drop(columns=['Basic Tokens'])
df_encoded.head()

,Disease,abnormal heart rhythm,abnormal menstruation,aching body,agitation,allergy,angina,anxiety,arm paralysis,arthritis,...,upset stomach,vision impairment,visual disturbances,vomiting,warm red skin over the affected joint,watering from eyes,weight gain,weight loss,wheezing,yellowing of eyes
0,Allergy,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1,Allergy,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,Allergy,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3,Allergy,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,Allergy,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [47]:
model_features = df_encoded.columns.to_list()

In [48]:
model_features

['Disease',
 'abnormal heart rhythm',
 'abnormal menstruation',
 'aching body',
 'agitation',
 'allergy',
 'angina',
 'anxiety',
 'arm paralysis',
 'arthritis',
 'autoimmune diseases',
 'bladder discomfort',
 'blood in sputum',
 'blood in stool',
 'blood in urine',
 'blurred and distorted vision',
 'blurred vision',
 'breathlessness',
 'bronchial asthma',
 'burning micturition',
 'changes in the mood',
 'chest pain',
 'chicken pox',
 'chills',
 'chronic kidney disease',
 'cloudy urine',
 'cold hands and feets',
 'cold sweat',
 'common cold',
 'confusion',
 'congestion',
 'continuous feel of urine',
 'continuous sneezing',
 'coronary artery disease',
 'cough',
 'dehydration',
 'depression',
 'diabetes',
 'diarrhea',
 'diarrhoea',
 'difficulty breathing',
 'difficulty speaking',
 'dizziness',
 'dry skin',
 'enlarged thyroid',
 'erectile dysfunction',
 'excessive hunger',
 'extreme tiredness',
 'facial paralysis',
 'falling over',
 'family history',
 'fast heart rate',
 'fatigue',
 'feeli

In [49]:
#model_features = df_encoded.columns.to_list()

In [50]:
model_features.remove("Disease")

In [51]:
model_features

['abnormal heart rhythm',
 'abnormal menstruation',
 'aching body',
 'agitation',
 'allergy',
 'angina',
 'anxiety',
 'arm paralysis',
 'arthritis',
 'autoimmune diseases',
 'bladder discomfort',
 'blood in sputum',
 'blood in stool',
 'blood in urine',
 'blurred and distorted vision',
 'blurred vision',
 'breathlessness',
 'bronchial asthma',
 'burning micturition',
 'changes in the mood',
 'chest pain',
 'chicken pox',
 'chills',
 'chronic kidney disease',
 'cloudy urine',
 'cold hands and feets',
 'cold sweat',
 'common cold',
 'confusion',
 'congestion',
 'continuous feel of urine',
 'continuous sneezing',
 'coronary artery disease',
 'cough',
 'dehydration',
 'depression',
 'diabetes',
 'diarrhea',
 'diarrhoea',
 'difficulty breathing',
 'difficulty speaking',
 'dizziness',
 'dry skin',
 'enlarged thyroid',
 'erectile dysfunction',
 'excessive hunger',
 'extreme tiredness',
 'facial paralysis',
 'falling over',
 'family history',
 'fast heart rate',
 'fatigue',
 'feeling confused'

In [52]:
X = df_encoded[model_features]
y = df_encoded["Disease"]

In [53]:
X

,abnormal heart rhythm,abnormal menstruation,aching body,agitation,allergy,angina,anxiety,arm paralysis,arthritis,autoimmune diseases,...,upset stomach,vision impairment,visual disturbances,vomiting,warm red skin over the affected joint,watering from eyes,weight gain,weight loss,wheezing,yellowing of eyes
0,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
184,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
185,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
186,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0


In [54]:
y

0       Allergy
1       Allergy
2       Allergy
3       Allergy
4       Allergy
         ...   
183    Migraine
184    Migraine
185    Migraine
186    Migraine
187    Migraine
Name: Disease, Length: 188, dtype: object

In [55]:
y_encoded = pd.get_dummies(y)

In [56]:
y_encoded

,Allergy,Arthritis,Autoimmune Diseases,Bronchial Asthma,Chicken pox,Chronic Kidney Disease,Common Cold,Coronary Artery Disease,Diabetes,Gastroenteritis,Hypertension,Hypothyroidism,Inflammatory Bowel Disease (IBD),Lower Urinary tract infection,Migraine,Pneumonia,Psoriasis,Stroke,Tuberculosis,Upper Urinary tract infection
0,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False
184,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False
185,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False
186,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False


In [57]:
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.25, random_state=42)
X_eval, X_test, y_eval, y_test = train_test_split(X_test, y_test, test_size=0.5, random_state=42)

rf_model = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=1, class_weight='balanced')

rf_model.fit(X_train, y_train)


In [58]:
rf_model = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=1, class_weight='balanced')

In [59]:
rf_model.fit(X_train, y_train)

RandomForestClassifier(class_weight='balanced', n_jobs=1, random_state=42)

In [60]:
y_pred = rf_model.predict(X_eval)

In [61]:
y_pred_labels = np.argmax(y_pred, axis=1)
y_eval_labels = np.argmax(y_eval.values, axis=1)

In [62]:
accuracy = accuracy_score(y_eval_labels, y_pred_labels)

In [63]:
print(f"Validation Acccuracy:  {accuracy}")

Validation Acccuracy:  1.0


In [64]:
y_test_pred = rf_model.predict(X_test)
test_accuracy = accuracy_score(y_test, y_test_pred)
print(f"Test Accuracy: {test_accuracy}")

Test Accuracy: 1.0


In [65]:
def predict_disease(symptoms, rf_model, mlb, encoder, df_encoded):
    # Step 1: Preprocess the symptoms
    def strip_to_basic_tokens(text):
        # Remove double spaces and underscores, then split by commas and lowercase the tokens
        text = re.sub(r'[_\s]+', ' ', text)
        tokens = [token.strip().lower() for token in text.split(',')]
        return tokens
    
    # Apply the tokenization process to the sample symptoms
    basic_tokens = strip_to_basic_tokens(symptoms)

    # Step 2: One-hot encode the tokens using the MultiLabelBinarizer (mlb) used during training
    one_hot_encoded_sample = mlb.transform([basic_tokens])
    
    # Create a DataFrame from the one-hot encoded sample
    one_hot_df = pd.DataFrame(one_hot_encoded_sample, columns=mlb.classes_)
    
    # Ensure that all columns (features) from training data are present (fill with 0 for missing columns)
    missing_columns = set(df_encoded.columns) - set(one_hot_df.columns)
    for col in missing_columns:
        one_hot_df[col] = 0
    
    # Reorder columns to match the original training DataFrame's order
    one_hot_df = one_hot_df[df_encoded.columns.difference(['Disease'])]

    # Step 3: Make prediction
    y_pred = rf_model.predict(one_hot_df)

    # Check if all values in `y_pred` are `False`
    if not y_pred.any():
        return "No_Matching"
    
    # Step 4: Decode the prediction (back to the disease label)
    predicted_class_index = np.argmax(y_pred)
    predicted_disease = encoder.inverse_transform([predicted_class_index])[0]
    
    return predicted_disease

# Example usage:
#sample_symptoms = "continuous_sneezing, shivering, chills, watering_from_eyes"
#sample_symptoms = "fatigue, weight_loss, restlessness, lethargy, irregular_sugar_level, blurred_and_distorted_vision, obesity, "
#sample_symptoms = "1, 2, 3, 4"
#sample_symptoms = "continuous_sneezing, chills, fatigue, cough,fever, headache, swelled_lymph_nodes, malaise, phlegm, throat_irritation, redness_of_eyes, sinus_pressure, runny_nose, congestion, chest_pain, loss_of_smell, muscle_pain"
sample_symptoms = "prolonged_cough,chest_pain,fatigue,yellowing_of_eyes,breathlessness,sweating,loss_of_appetite,phlegm,night_sweats,high_fever"

predicted_disease = predict_disease(sample_symptoms, rf_model, mlb, encoder, df_encoded)

print(f"The predicted disease for the symptoms '{sample_symptoms}' is: {predicted_disease}")


The predicted disease for the symptoms 'prolonged_cough,chest_pain,fatigue,yellowing_of_eyes,breathlessness,sweating,loss_of_appetite,phlegm,night_sweats,high_fever' is: Tuberculosis


In [66]:
import pickle

# Save the Random Forest model
with open("Models/rf_model.pkl", "wb") as model_file:
    pickle.dump(rf_model, model_file)

# Save the label encoder
with open("Models/label_encoder.pkl", "wb") as le_file:
    pickle.dump(encoder, le_file)


# Save the MultiLabelBinarizer
with open("Models/mlb.pkl", "wb") as mlb_file:
    pickle.dump(mlb, mlb_file)

print("Model, Label Encoder, and MultiLabelBinarizer saved successfully!")


Model, Label Encoder, and MultiLabelBinarizer saved successfully!
